Jon V

6/3/18

In [39]:
from ase.spacegroup import crystal
from ase.build import *
import numpy as np
import matplotlib.pyplot as plt
import os
import ase
import json

### Choose inputs.

In [40]:
# define scf parameters
pref = 'benzene'
pseudo_dir = '/n/home03/jonpvandermause/qe-6.2.1/pseudo'
outdir='/n/home03/jonpvandermause/Cluster/Benzene_SCF'

# specify locations of QE codes
npool = 1 # number of k-pt pools
mult = 32
ncpu = npool * mult # number of cores
nodes = int(np.ceil(ncpu/32)) # number of nodes
pw_loc = '/n/home03/jonpvandermause/qe-6.2.1/bin/pw.x'
in_name = 'benzene.scf.in'
out_name = 'benzene.scf.out'
sh_name = 'benzene.sh'
partition = 'kozinsky'
memory = 1000
email = 'jonathan_vandermause@g.harvard.edu'

### Define helper functions.

In [41]:
def write_file(fname, text):
    with open(fname, 'w') as fin:
        fin.write(text)

### Make scf file.

In [42]:
def get_scf_text(pref, pseudo_dir, outdir):
    
    scf_text = """&control
    calculation = 'scf'
    prefix='{0}'    
    pseudo_dir = '{1}',
    outdir='{2}'
 /
 &system
    ibrav = 6,
    celldm(1) = 32,
    celldm(3) = 0.83,
    nat = 12,
    ntyp = 2,
    ecutwfc = 25,
    ecutrho = 250,
 /  
 &electrons
    diagonalization='david'
    mixing_mode = 'plain'
    mixing_beta = 0.7
    conv_thr =  1.0d-10
    !tqr=.true.
 /
 &ions
 /
ATOMIC_SPECIES
C 12.01  C.pz-rrkjus.UPF
H 1.0    H.pz-rrkjus.UPF
ATOMIC_POSITIONS {{Angstrom}}
C        5.633200899   6.320861303   5.
C        6.847051545   8.422621957   5.
C        8.060751351   7.721904557   5.
C        8.060707879   6.320636665   5.
C        6.846898786   5.620067381   5.
C        5.633279551   7.722134449   5.
H        6.847254360   9.512254789   5.
H        9.004364510   8.266639340   5.
H        9.004297495   5.775895755   5.
H        6.846845929   4.530522778   5.
H        4.689556006   5.776237709   5.
H        4.689791688   8.267023318   5.
K_POINTS {{gamma}}""".format(pref, pseudo_dir, outdir)
    
    return scf_text

In [43]:
# make text
scf_text = get_scf_text(pref, pseudo_dir, outdir)
write_file(in_name, scf_text)

### Make sh file.

In [44]:
def make_sh_text(ncpu, nodes, partition, memory, email, pw_loc, npool, in_file, out_file):

    sh_text = """#!/bin/sh
#SBATCH -n {0}
#SBATCH -N {1}
#SBATCH -t 1-00:00
#SBATCH -e test.err
#SBATCH -p {2}
#SBATCH -o test.out
#SBATCH --mem-per-cpu={3}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={4}

module load gcc/4.9.3-fasrc01 openmpi/2.1.0-fasrc01
module load python/3.6.3-fasrc01

MPI="mpirun"
PW="{5}"

$MPI $PW -npool {6} < {7} > {8}
""".format(ncpu, nodes, partition, memory, email, pw_loc, npool, in_file, out_file)
    
    return sh_text

In [45]:
# make sh text
sh_text = make_sh_text(ncpu, nodes, partition, memory, email, pw_loc, npool, in_name, out_name)
write_file(sh_name, sh_text)